# Convex Minimization Example
This script demonstrates the convex minimization problem in the proposed algorithm.

In [1]:
import numpy as np
import cvxpy as cp

In [309]:
# Setup
ACT_DIM = 2 # action space dimensions
ETA = 1e-8 # used to enforce strict inequality

beta = cp.Parameter() # (> 1) constraint on maximum policy ratio

# safe policy (cvxpy parameters)
mu_safe = cp.Parameter(ACT_DIM)
sigma_safe = cp.Parameter(ACT_DIM, pos=True)

# task policy (cvxpy parameters)
mu_task = cp.Parameter(ACT_DIM)
sigma_task = cp.Parameter(ACT_DIM, pos=True)

# projected policy (cvxpy variables)
mu_proj = cp.Variable(ACT_DIM)
sigma_proj = cp.Variable(ACT_DIM, pos=True)

In [325]:
# Problem definition
objective = cp.Minimize(-2*cp.sum(cp.log(sigma_proj)) +  cp.sum_squares(sigma_proj/sigma_task) + cp.sum_squares((mu_proj - mu_task)/sigma_task))

constraints = []

constraint_1 = sigma_proj + ETA <= sigma_safe
constraints.append(constraint_1)

sum_term = 0    # we define the sum term like this so that we can use quad_over_lin and thus allow the problem to be DCP (standard quotient operator is not DCP)
for i in range(0, ACT_DIM):
    sum_term += cp.quad_over_lin(mu_proj[i] - mu_safe[i],cp.square(sigma_safe[i]) - cp.square(sigma_proj[i]))   # note that numerator gets squared (which we want)

constraint_2 = cp.sum(cp.log(sigma_safe)) - cp.sum(cp.log(sigma_proj)) + (1/2)*sum_term <= cp.log(beta)
constraints.append(constraint_2)

prob = cp.Problem(objective, constraints)

# Check problem is DCP (or DPP)
CHECK_DPP = True
print(objective.is_dcp(dpp=CHECK_DPP))
print(constraint_1.is_dcp(dpp=CHECK_DPP))
print(constraint_2.is_dcp(dpp=CHECK_DPP))
print(prob.is_dcp(dpp=CHECK_DPP))

False
True
False
False


In [328]:
# Generate examples
beta.value = 1.1 # (> 1) constraint on maximum policy ratio

STRAY_FACTOR = 0.05 # factor on how far the task policy strays from the safe policy

# safe policy
mu_safe.value = np.random.randn(ACT_DIM)
sigma_safe.value = np.random.exponential(size=ACT_DIM) # > 0

# task policy
mu_task.value = mu_safe.value + STRAY_FACTOR*np.random.randn(ACT_DIM) # put task policy mean somewhere fairly close to safe policy mean
sigma_task.value = sigma_safe.value - np.minimum(STRAY_FACTOR*sigma_safe.value*np.random.exponential(size=ACT_DIM),sigma_safe.value-ETA)  # put task policy stdev somewhere fairly close to but less than safe policy stdev (and > 0)

In [329]:
# Solve problem
result = prob.solve()
print(mu_safe.value, sigma_safe.value)
print(mu_task.value, sigma_task.value)
print(mu_proj.value, sigma_proj.value)

[-0.10747857  0.07784673] [0.15652598 2.10122467]
[-0.14160945  0.04111572] [0.15570198 2.08318979]
[-0.12167259  0.07784672] [0.15102287 2.1012246 ]


In [273]:
# Failed attempt (is not DCP)

alpha = cp.Variable(ACT_DIM, pos=True) # slack variable used to make problem DCP
log_mu_difference = cp.Variable(ACT_DIM, pos=True) # slack variable used to make problem DCP

objective = cp.Minimize(-2*cp.sum(cp.log(sigma_proj)) +  cp.sum_squares(sigma_proj/sigma_task) + cp.sum_squares((mu_proj - mu_task)/sigma_task))
constraints = []
constraint_1 = sigma_proj + eta <= sigma_safe
constraints.append(constraint_1)
# constraint_2 = cp.exp(1/2*cp.square(mu_proj - mu_safe)/(cp.square(sigma_safe) - cp.square(sigma_proj))) <= beta*sigma_proj/sigma_safe
# constraint_2 = cp.sum(cp.log(sigma_safe)) - cp.sum(cp.log(sigma_proj)) + (1/2)*cp.sum(cp.square(mu_proj-mu_safe)) <= cp.log(beta)
constraint_2 = cp.sum(cp.log(sigma_safe)) - cp.sum(cp.log(sigma_proj)) + (1/2)*cp.sum(alpha) <= cp.log(beta) # need slack variable alpha to handle the summands of the sum term
constraints.append(constraint_2)
# constraint_3 = 2*cp.log(mu_proj - mu_safe) - cp.log(cp.square(sigma_safe) - cp.square(sigma_proj)) <= cp.log(alpha)
constraint_3 = 2*log_mu_difference - cp.log(cp.square(sigma_safe) - cp.square(sigma_proj)) <= cp.log(alpha) # need to add slack variable log_mu_difference to make DCP (convex <= concave)
constraints.append(constraint_3)
constraint_4 = mu_proj - mu_safe <= cp.exp(log_mu_difference) # not DCP because we need convex <= concave but the RHS is convex (taking logs leads to a similar problem)
constraints.append(constraint_4)
# constraints = [cp.prod(cp.exp(1/2*cp.square(mu_proj - mu_safe)/(cp.square(sigma_safe) - cp.square(sigma_proj)))) <= beta*cp.prod(sigma_proj/sigma_safe), sigma_proj + eta <= sigma_safe]
prob = cp.Problem(objective, constraints)

print(objective.is_dcp())
print(constraint_1.is_dcp())
print(constraint_2.is_dcp())
print(constraint_3.is_dcp())
print(constraint_4.is_dcp())

True
True
True
True
False
